In [1]:


import astropy
print('astropy',astropy.__version__ )
from spectral_cube import SpectralCube      # This is a handy package for working with 3D data cubes
from spectral_cube import LazyMask
from astropy.coordinates import SkyCoord
from reproject import reproject_interp      
from reproject.mosaicking import find_optimal_celestial_wcs 
import regions
import reproject
print('reproject',reproject.__version__)
import spectral_cube
print('spectral_cube',spectral_cube.__version__)
import numpy as np                          
import pylab                                
import matplotlib 
import matplotlib.gridspec as gridspec                                                                                             
import scipy
import astropy.io.fits as fits                                                          
from astropy.wcs import WCS                 
from astropy import units as u              
import pyspeckit as psk   
import astrodendro
import matplotlib.colors as colors
from matplotlib import pyplot as plt
# Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%matplotlib inline
from astropy.convolution import Gaussian1DKernel
#%matplotlib widget
import radio_beam
import copy
from tqdm import tqdm

astropy 5.1.dev153+gb740594dc
reproject 0.8
spectral_cube 0.6.1.dev22+g003ef16


In [4]:
!pip install tqdm 

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)   
    #header = fits.getheader(file)
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    #if i ==2:
    if i==0 or i ==1 or i ==2:

        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=100
        if raDP1[1]<0:
            raDP1[1]=100
        sc = sc#[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms.spectral_slab(0. *u.km / u.s, 500. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=3*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/3.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/3.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/3.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/3.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.max(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
        if i ==0:
            vel = np.arange(0,501,2.5)*u.km/u.s


            fwhm_factor = np.sqrt(8*np.log(2))

            NGC_HCO_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
            del sc_K_kms

            NGC_HCO_Plus_Reproject_Interp = NGC_HCO_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
            del NGC_HCO_reprojectGauss
            NGC_HCO_Plus_Reproject_Interp.write('NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits',overwrite=True)
            del NGC_HCO_Plus_Reproject_Interp

        if i==1:
            fwhm_factor = np.sqrt(8*np.log(2))

            NGC_HCN_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
            del sc_K_kms

            NGC_HCN_Plus_Reproject_Interp = NGC_HCN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
            del NGC_HCN_reprojectGauss

            NGC_HCN_Plus_Reproject_Interp.write('NGC253_HCN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)
            del NGC_HCN_Plus_Reproject_Interp

        if i==2:

            NGC_H13CN = sc_K_kms#This one needs reprojection


del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN.allow_huge_operations=True

NGC_HCO_Plus_Reproject_Interp = SpectralCube.read('NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits')
HCOPH = NGC_HCO_Plus_Reproject_Interp.header


print(NGC_H13CN)


print(type(NGC_H13CN))


print(type(NGC_H13CN[0]))


print(type(NGC_H13CN[0][0]))
fwhm_factor = np.sqrt(8*np.log(2))

NGC_HCN_reprojectGauss = NGC_H13CN.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_H13CN
      

NGC_HC13N_reproject_interp = NGC_HCN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCN_reprojectGauss


NGC_H13CN_reproject = NGC_HC13N_reproject_interp.reproject(HCOPH,use_memmap=True)  
del NGC_HC13N_reproject_interp #rem from mem
del HCOPH


NGC_H13CN_reproject.write('NGC_H13CN.cube.reproject.mJy-kms.pbcor.fits',overwrite=True)      
del NGC_H13CN_reproject         
print("done")        

[288, 330] [641, 662]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=0.0 deg
65730.9765625 mJy / beam
0.0 -1.2796754075622423
1.0315638844900253e-09 2.5820095130479396e-10
86516.88581681426 mJy / beam
(201, 938, 1073)
-1.637022271802352e-05 11.895693160611499 B
1.637022271802352e-05 -25.29536924799503 c
1006 795
84434.48905472482 mJy / beam
1
[288, 330] [641, 662]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=0.0 deg
73385.2578125 mJy / beam
0.0 -1.2796754075622423
1.0315638844900253e-09 2.5820095130479396e-10
95381.63426648549 mJy / beam
(201, 938, 1073)
-1.637022271802352e-05 11.895693160611499 B
1.637022271802352e-05 -25.29536924799503 c
1006 795
93839.1235341746 mJy / beam
1
[288, 330] [641, 662]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=

In [ ]:
#The first three cubes haves been reporjected

In [55]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits',"NGC253.band3.CO_1-0.PB_edge_masked.fits"]     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    if i==0 or i ==3:

        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=0
        if raDP1[1]<0:
            raDP1[1]=0
        sc = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms.spectral_slab(0. *u.km / u.s, 500. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=5*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.max(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
    if i==0:
        vel = np.arange(0,501,2.5)*u.km/u.s
        
        fwhm_factor = np.sqrt(8*np.log(2))

        NGC_HCN_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
        del sc_K_kms

        NGC_HCN_Plus_Reproject_Interp = NGC_HCN_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
        del NGC_HCN_reprojectGauss

        NGC_HCN_Plus_Reproject_Interp.write('NGC253_HCO+.cube_5pc_beam.reproject.mJy-kms.pbcor.fits',overwrite=True)
        del NGC_HCN_Plus_Reproject_Interp
            
            
        
    if i==3:
        
        NGC_HCN_J1 = sc_K_kms


del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCN_J1.allow_huge_operations=True
NGC_HCO_Plus = SpectralCube.read('NGC253_HCO+.cube_5pc_beam.reproject.mJy-kms.pbcor.fits')
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'  #Change from F2F because it seems to be the same thing. See https://www.aanda.org/articles/aa/pdf/2006/05/aa3818-05.pdf which says that it means "Sampled in freq, expressed in freq". Side note, some comments in the header may be incorrect, such as incorrect units (comments say m/s where it should be km/s)


del NGC_HCO_Plus #rem from mem
print(np.max(NGC_HCN_J1))
NGC_HCN_J1_reproject = NGC_HCN_J1.reproject(HCOPH,use_memmap=True)  
print(np.max(NGC_HCN_J1_reproject))
del NGC_HCN_J1 #rem from mem
del HCOPH



fwhm_factor = np.sqrt(8*np.log(2))

NGC_HCN_J1_reprojectGauss = NGC_HCN_J1_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
print(np.max(NGC_HCN_J1_reprojectGauss ))
del NGC_HCN_J1_reproject
      

NGC_HCN_J1_reproject_interp = NGC_HCN_J1_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
print(np.max(NGC_HCN_J1_reproject_interp ))
del NGC_HCN_J1_reprojectGauss


NGC_HCN_J1_reproject_interp.write('NGC_HCN_J1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_HCN_J1_reproject_interp
print("done")        

[288, 330] [641, 662]
8.18511135901176e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.29466400892442335 arcsec BMIN=0.29466400892442335 arcsec BPA=0.0 deg
65730.9765625 mJy / beam
0.0 -1.2796754075622423
5.319310953893119e-09 4.545948020707888e-09
166828.05250763585 mJy / beam
(201, 332, 353)
-1.6370222718023522e-05 11.891268833495984 B
1.6370222718023522e-05 -25.29078609484757 c
331 281
164779.18441856545 mJy / beam
1
[244, 349] [813, 885]
8.18511135901176e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.29466400892442335 arcsec BMIN=0.29466400892442335 arcsec BPA=0.0 deg
0.02085256204009056 Jy / beam
0.0 1.4368850473684927
4.024642316878407e-09 1.8532232358014941e-09
0.03092486820909891 Jy / beam
(153, 536, 569)
-1.6370222718023522e-05 11.89125850196911 B
1.6370222718023522e-05 -25.290783312351873 c
331 282
0.0307357045661906 Jy / beam
1
0.0307357045661906 Jy / beam
0.030235410401362316 Jy / beam
0.029407726090408023 Jy / beam
0.029407726090408023 Jy / beam
done

In [56]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits',"NGC253.band3.CO_1-0.PB_edge_masked.fits"]     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    if i==0 or i ==4:

        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=0
        if raDP1[1]<0:
            raDP1[1]=0
        sc = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms.spectral_slab(0. *u.km / u.s, 500. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=5*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.max(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
    if i==0:
        vel = np.arange(0,501,2.5)*u.km/u.s
        
        NGC_HCO_Plus = SpectralCube.read('NGC253_HCO+.cube_5pc_beam.reproject.mJy-kms.pbcor.fits')
    if i==4:
        
        NGC_HCO_J1 = sc_K_kms


del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_HCO_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'  #Change from F2F because it seems to be the same thing. See https://www.aanda.org/articles/aa/pdf/2006/05/aa3818-05.pdf which says that it means "Sampled in freq, expressed in freq". Side note, some comments in the header may be incorrect, such as incorrect units (comments say m/s where it should be km/s)


del NGC_HCO_Plus #rem from mem

NGC_HCO_J1_reproject = NGC_HCO_J1.reproject(HCOPH,use_memmap=True)  
del NGC_HCO_J1 #rem from mem
del HCOPH



fwhm_factor = np.sqrt(8*np.log(2))

NGC_HCO_J1_reprojectGauss = NGC_HCO_J1_reproject.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_HCO_J1_reproject
      

NGC_HCO_J1_reproject_interp = NGC_HCO_J1_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_HCO_J1_reprojectGauss


NGC_HCO_J1_reproject_interp.write('NGC_HCOP_J1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_HCO_J1_reproject_interp
print("done")        

[288, 330] [641, 662]
8.18511135901176e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.29466400892442335 arcsec BMIN=0.29466400892442335 arcsec BPA=0.0 deg
65730.9765625 mJy / beam
0.0 -1.2796754075622423
5.319310953893119e-09 4.545948020707888e-09
166828.05250763585 mJy / beam
(201, 332, 353)
-1.6370222718023522e-05 11.891268833495984 B
1.6370222718023522e-05 -25.29078609484757 c
331 281
164779.18441856545 mJy / beam
1
[244, 349] [813, 885]
8.18511135901176e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.29466400892442335 arcsec BMIN=0.29466400892442335 arcsec BPA=0.0 deg
0.02086353488266468 Jy / beam
0.0 1.4377583330805506
4.050680912959141e-09 1.8960168557714113e-09
0.030931089994657403 Jy / beam
(154, 536, 569)
-1.6370222718023522e-05 11.89125850196911 B
1.6370222718023522e-05 -25.290783312351873 c
331 282
0.03081789423120663 Jy / beam
1
done


In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
        
    if i==5:
        NGC_H13CN_J1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_H13CN_J1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

NGC_H13CN_J1_reproject = NGC_H13CN_J1.reproject(HCOPH,use_memmap=True)  
del NGC_H13CN_J1 #rem from mem
del HCOPH
NGC_H13CN_J1_reproject_interp = NGC_H13CN_J1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del NGC_H13CN_J1_reproject
NGC_H13CN_J1_reproject_interp.write('NGC_H13CN_J1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)  
del NGC_H13CN_J1_reproject_interp        
    
     
print("done")             

In [3]:
files=['NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
files=['NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits',"NGC253.band3.CO_1-0.PB_edge_masked.fits"]     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    if i ==6:

        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=100
        if raDP1[1]<0:
            raDP1[1]=100
        sc = sc#[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms.spectral_slab(0. *u.km / u.s, 500. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=3*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/3.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/3.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/3.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/3.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.max(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
        
    if i ==0:
        vel = np.arange(0,501,2.5)*u.km/u.s
        
        
    if i==6:
        NGC_CO_J3_2 = sc_K_kms

        

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J3_2.allow_huge_operations=True
NGC_HCO_Plus =  SpectralCube.read(files[0])
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem



fwhm_factor = np.sqrt(8*np.log(2))

CO32_reprojectGauss = NGC_CO_J3_2.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_CO_J3_2

CO32_reproject_interp = CO32_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 

del CO32_reprojectGauss
CO32_reproject_interp = CO32_reproject_interp.hdu

NGC_CO_J3_2_reproject_interp = SpectralCube.read(CO32_reproject_interp)

NGC_CO_J3_2_reproject_interp.allow_huge_operations=True


NGC_CO_J3_2_reproject = NGC_CO_J3_2_reproject_interp.reproject(HCOPH,use_memmap=True)  
del NGC_CO_J3_2_reproject_interp #rem from mem
del HCOPH

print(NGC_CO_J3_2_reproject.world[:,0,0][0])
NGC_CO_J3_2_reproject.write('NGC_CO_J3_2.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del NGC_CO_J3_2_reproject  

print("done")         

[580, 509] [1099, 997]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=0.0 deg
0.3329935669898987 Jy / beam
0.0 0.0
4.881760308255835e-11 4.881760308255835e-11
0.3396753426528447 Jy / beam
(81, 1536, 1800)
-1.637022271802352e-05 11.897318120737346 B
1.637022271802352e-05 -25.295586369447484 c
1148 886
0.33741369265763166 Jy / beam
1
[  0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90.   92.5  95.   97.5 100.  102.5 105.  107.5 110.  112.5 115.  117.5
 120.  122.5 125.  127.5 130.  132.5 135.  137.5 140.  142.5 145.  147.5
 150.  152.5 155.  157.5 160.  162.5 165.  167.5 170.  172.5 175.  177.5
 180.  182.5 185.  187.5 190.  192.5 195.  197.5 200.  202.5 205.  207.5
 210.  212.5 215.  217.5 220.  222.5 225.  227.5 230.  232.5 235

In [6]:
#Just oversample co32
#files=['ngc253_co32_hires_common.fits']
files = ["NGC253.band7.TP+12m-mid+12m-high.CO_3-2.image.pbcor.fits"]
      



for i in range(0,300):
    
    sc = SpectralCube.read(files[0])  
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    #if i >0:
    if True:

        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=100
        if raDP1[1]<0:
            raDP1[1]=100
        if(i*30>len(sc)):
            print("End Loop at",i)
            break
            
        sc = sc[i*30:i*30+30,:,:]#[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms.spectral_slab(0. *u.km / u.s, 500. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        
        HI = sc.header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=3*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.nanmax(sc_K_kms),np.shape(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
        
    
    vel = np.arange(0,501,2.5)*u.km/u.s
        
        
    
    NGC_CO_J3_2 = sc_K_kms
        
    del cen_p1
    del cen_p2
    del scW
    del raDP1
    del raDP2
    NGC_CO_J3_2.allow_huge_operations=True

    
    fwhm_factor = np.sqrt(8*np.log(2))

    CO32_reprojectGauss = NGC_CO_J3_2.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
    del NGC_CO_J3_2
    
    NGC_CO_J3_2_reproject_interp = NGC_CO_J3_2.spectral_interpolate(spectral_grid=vel) # Match velocities to 0 501 range 



    NGC_CO_J3_2_reproject_interp.allow_huge_operations=True

    
    

    print(np.shape(NGC_CO_J3_2))
    NGC_CO_J3_2.write(("3pc_beam_"+str(i)+'_NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits'),overwrite=True)      
    del NGC_CO_J3_2

        



print("done")         

[297, 338] [650, 670]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=0.0 deg
37.86989212036133 K
0.0 -1.3107422682477414
1.0466256833459316e-09 2.431391524488885e-10
35.63169553804524 K
(30, 954, 1090)
-9.822133630814112e-06 11.895831424165491 B
9.822133630814112e-06 -25.295480351926876 c
1704 1348
35.2687370229037 (30, 1348, 1704)
1
(30, 1348, 1704)
[297, 338] [650, 670]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.17679840535465402 arcsec BMIN=0.17679840535465402 arcsec BPA=0.0 deg
86.58851623535156 K
0.0 -1.3107422682477414
1.0466256833459316e-09 2.431391524488885e-10
84.11183436190109 K
(30, 954, 1090)
-9.822133630814112e-06 11.895831424165491 B
9.822133630814112e-06 -25.295480351926876 c
1704 1348
83.64330628110908 (30, 1348, 1704)
1
(30, 1348, 1704)
[297, 338] [650, 670]
4.911066815407056e-05 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=0.176798405354

In [2]:
#Splice CO

#mask = []
#Full_Mask = LazyMask(function = duh,data= mask, wcs = rewcs)
vel = np.arange(0,501,2.5)*u.km/u.s    
for i in range(0,9):
    sc=SpectralCube.read(("3pc_beam_"+str(i)+'_NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits'))      
    print(sc.mask,type(sc.mask))
        
    if i==0:
        reheader = sc.header
        rewcs = sc.wcs
    
    
    print(type(sc))
    if i ==0:
        scW=SpectralCube.read(("3pc_beam_"+str(i)+'_NGC_CO_J3_2.cube.reproject.K-kms.pbcor.fits'))      
        mask = scW.mask.include()
    else:


        if i ==1:
            scW = np.concatenate((scW[:].hdu.data,sc[:].hdu.data),dtype = type(sc))
            print(type(scW))
            mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
        else:
            scW = np.concatenate((scW[:],sc.hdu.data[:]),dtype = type(sc))
            mask = np.concatenate((mask[:],sc[:].mask.include()),dtype = type(sc[:].mask.include()))
        
            
    print(np.shape(scW))

def duh(lol):
    gp = np.where(lol!=np.nan)
    lol[gp]=True
    return lol
reheader["NAXIS3 "] = len(scW)
 
print(type(scW),sc[0][0][0].unit)

#print(mask)

Full_Mask = LazyMask(function = duh,data = mask, wcs = rewcs)
print(Full_Mask,np.shape(Full_Mask),Full_Mask.include())
scWsc = SpectralCube(data = scW,wcs = rewcs, header = reheader, mask = Full_Mask)

scWsc.allow_huge_operations=True
print(scWsc[0][0][0])
scWsc = scWsc*sc[0][0][0].unit

fwhm_factor = np.sqrt(8*np.log(2))

print(type(scWsc),np.shape(scWsc),scWsc.header,rewcs)
scWsc.allow_huge_operations=True





#scWsc = scWsc.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""

print(np.shape(scWsc),np.shape(scWsc.mask))
print(scWsc.mask,scWsc.mask.include())
scWsc = scWsc.spectral_interpolate(spectral_grid=vel) # Match velocities to 0 501 range 


scWsc.write("NGC_Spliced_Reprojected_Whole_CO_32_No_Gauss_Corr.fits",overwrite=True)


print("done")         

<spectral_cube.masks.LazyMask object at 0x7fbd6807ab20> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(30, 1348, 1704)
<spectral_cube.masks.LazyMask object at 0x7fbcd1a15490> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
<class 'numpy.ndarray'>
(60, 1348, 1704)
<spectral_cube.masks.LazyMask object at 0x7fbcf2a6ee50> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(90, 1348, 1704)
<spectral_cube.masks.LazyMask object at 0x7fbcf2862a00> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(120, 1348, 1704)
<spectral_cube.masks.LazyMask object at 0x7fbd6807a820> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(150, 1348, 1704)
<spectral_cube.masks.LazyMask object at 0x7fbcd1a022b0> <class 'spectral_cube.masks.LazyMask'>
<class 'spectral_cube.spectral_cube.SpectralCube'>
(180, 1348, 1

In [2]:
#Subtract the continuum
cont = "ngc253_850_micron_dust.fits"
scCont = spectral_cube.Projection.from_hdu(fits.open(cont)[0])

path='Cropped_NGC_Spliced_Reprojected_Whole_CO_32.fits'


sc =  SpectralCube.read(path).with_spectral_unit(u.km/u.s,velocity_convention="radio").unmasked_copy()
sc.allow_huge_operations=True
sc = sc.to(u.Jy/u.beam)
data=sc.hdu.data

cont_dat = scCont.reproject(sc.moment0().header).hdu.data
for i in range(len(sc)):
    print(i)
    
    data[i] -= cont_dat

Subtract = SpectralCube(data = data,wcs = sc.wcs, header = sc.header)
Subtract.allow_huge_operations=True
Subtract = Subtract*u.Jy/u.beam

Subtract.write("Subtracted_Cropped_CO_3_2_NGC253.fits",overwrite=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192


In [13]:
#DO THIS FOR THE WHOLE CUBE NOW


###################################################################################################
# align orientation
###################################################################################################

cube4 = SpectralCube.read("Cropped_NGC_Spliced_Reprojected_Whole_CO_32.fits")
cube4 = cube4[:,300:len(cube4[0])-300,300:len(cube4[0][0])-400]#
cube4.allow_huge_operations = True


# set reference pixels to galaxy center
###################################################################################################

# Spectalcube does not allow to write header data! Need to use astropy instead.
cube5 = cube4.hdu

# get center coordinates
center = SkyCoord('00h47m33.134s' ,'-25d17m19.68s',frame='icrs')
center_x = center.ra
center_y = center.dec

# get image coordinates
spine_x = cube4.world[0,0,:][2]
spine_y = cube4.world[0,:,0][1]
del cube4
# find pixel index of center
diff_x = [np.abs(x-center_x).value for x in spine_x]
diff_y = [np.abs(y-center_y).value for y in spine_y]
crpix1 = np.argmin(diff_x)
crpix2 = np.argmin(diff_y)

# update header
cube5.header['crpix1'] = crpix1
cube5.header['crpix2'] = crpix2
cube5.header['crval1'] = 0.
cube5.header['crval2'] = 0.


# rotate NGC253
###################################################################################################
gal = "NGC253"
if gal=='NGC253':

    # rotation matrix for 33 degree
    rotation_matrix = {'PC1_1':  8.386705679454E-01,
                       'PC2_1': -5.446390350150E-01,
                       'PC3_1':  0.000000000000E+00,
                       'PC1_2': 5.446390350150E-01,
                       'PC2_2':  8.386705679454E-01,
                       'PC3_2':  0.000000000000E+00,
                       'PC1_3':  0.000000000000E+00,
                       'PC2_3':  0.000000000000E+00,
                       'PC3_3':  1.000000000000E+00}

    reheader = copy.deepcopy(cube5.header)
    for key,val in rotation_matrix.items():
        reheader[key] = val

    # rotate by regridding onto rotated header
    cube5 = SpectralCube.read(cube5)
    cube5.allow_huge_operations = True
    cube6 = cube5.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
    del cube5
    cube6 = cube6.hdu

    # remove roation matrix
    for key in rotation_matrix.keys():
        try:
            del cube6.header[key]
        except:
            pass

else:
    cube6 = cube5


# convert to pc grid
###################################################################################################

cdelt1 = u.Quantity(str(cube6.header['cdelt1'])+cube6.header['cunit1'])
cdelt2 = u.Quantity(str(cube6.header['cdelt2'])+cube6.header['cunit2'])

cdelt1_pc = cdelt1*np.pi/180*3.5*10**6*u.pc/u.deg
cdelt2_pc = cdelt2*np.pi/180*3.5*10**6*u.pc/u.deg

cube6.header['cdelt1'] = cdelt1_pc.value
cube6.header['cdelt2'] = cdelt2_pc.value
cube6.header['cunit1'] = 'pc'
cube6.header['cunit2'] = 'pc'
cube6.header['ctype1'] = 'linear'
cube6.header['ctype2'] = 'linear'


# extract matching subcube (take inclination into account)
###################################################################################################

fov_x = 800*u.pc#CO['fov'][0]
fov_y = 400*u.pc#CO['fov'][1]

# get cube info
center_x = int(cube6.header['crpix1'])
center_y = int(cube6.header['crpix2'])
cdelt_x = u.Quantity(str(np.abs(cube6.header['cdelt1']))+cube6.header['cunit1'])
cdelt_y = u.Quantity(str(np.abs(cube6.header['cdelt2']))+cube6.header['cunit2'])

# calculate pixel position of subcube
npix_x = int(np.round(fov_x/2./cdelt_x))
npix_y = int(np.round(fov_y/2./cdelt_y))
cutout_x = [center_x-npix_x, center_x+npix_x]
cutout_y = [center_y-npix_y, center_y+npix_y]

# check image dimensions
grow_x1 = 0-cutout_x[0] if cutout_x[0]<0 else 0
grow_x2 = cutout_x[1]-cube6.header['naxis1'] if cutout_x[1]>cube6.header['naxis1'] else 0
grow_x  = np.max([grow_x1, grow_x2])

grow_y1 = 0-cutout_y[0] if cutout_y[0]<0 else 0
grow_y2 = cutout_y[1]-cube6.header['naxis2'] if cutout_y[1]>cube6.header['naxis2'] else 0
grow_y  = np.max([grow_y1, grow_y2])

if grow_x>0 or grow_y>0:
    print("Growing image to allow cutting out symmetric subimage.")
    shape = cube6.shape
    grown = np.full((shape[0],shape[1]+2*grow_y,shape[2]+2*grow_x),np.nan)
    for v,V in tqdm(enumerate(cube6.data)):
        for y,Y in enumerate(V):
            for x,X in enumerate(Y):
                grown[v,y+grow_y,x+grow_x] = X
    cube6.data = grown
    cutout_x = [x+grow_x for x in cutout_x]
    cutout_y = [y+grow_y for y in cutout_y]

# cut out subimage
cube7   = cube6.data[:,cutout_y[0]:cutout_y[1],cutout_x[0]:cutout_x[1]]
header7 = copy.deepcopy(cube6.header)

# update reference pixels
header7['crpix1'] = npix_x
header7['crpix2'] = npix_y

#cube7 = fits.open("CO_3_2_NGC_Matched.fits")[0].data
#header7 =fits.open("CO_3_2_NGC_Matched.fits")[0].header
#WCS = astropy.wcs.WCS(header7)

fits.writeto(("CO_3_2_NGC_Matched.fits"),
             data      = cube7,
             header    = header7,
             overwrite = True             
            )

#cube8= SpectralCube(data=cube7,header=header7,wcs=WCS)




ValueError: No celestial axes found in WCS

In [ ]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits']     
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==7:
        CCH_N1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
CCH_N1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

CCH_N1_reproject =CCH_N1.reproject(HCOPH,use_memmap=True)  
del CCH_N1 #rem from mem
del HCOPH
CCH_N1_reproject_interp = CCH_N1_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del CCH_N1_reproject
CCH_N1_reproject_interp.write('CCH_N1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)   
del CCH_N1_reproject_interp         
print("done")            

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3.pbcor.fits','NGC253.H13CN_4-3.pbcor.fits','HCN_J1-0.cube.fits','HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits',"HC3N_11_10.fits"])
      
dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==8:
        HC3N_J11 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
HC3N_J11.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

HC3N_J11_reproject =HC3N_J11.reproject(HCOPH,use_memmap=True)  
del HC3N_J11 #rem from mem
del HCOPH
HC3N_J11_reproject_interp = HC3N_J11_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del HC3N_J11_reproject
HC3N_J11_reproject_interp.write('HC3N_J11.cube.reproject.Jy-kms.pbcor.fits',overwrite=True) 
del HC3N_J11_reproject_interp          
print("done")        

In [ ]:

      
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits', "H13CO_Plus_1_0.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms
        
    if i==1:
        H13CoPJ1_0 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
H13CoPJ1_0.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
print(HCOPH)
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

H13CoPJ1_0_reproject = H13CoPJ1_0.reproject(HCOPH,use_memmap=True)  
del H13CoPJ1_0 #rem from mem
del HCOPH
H13CoPJ1_0_reproject_interp = H13CoPJ1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del H13CoPJ1_0_reproject
H13CoPJ1_0_reproject_interp.write('H13CoPJ1_0.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)      
del H13CoPJ1_0_reproject_interp        
           
print("done")        

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy.pbcor.fits', "SiO_2_1.fits"])


for i,file in enumerate(files):
                    
    sc = SpectralCube.read(file)  
    
    cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')
    cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')
    scW = sc.wcs[:][:][0]
    raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
    raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
    subcube = sc[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube
    
    
    #Put in the right system
    sc_kms = subcube.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
    sc_slab = sc_kms.spectral_slab(-1. *u.km / u.s, 501. *u.km / u.s)  # Crop out velocities we don't care about    
    sc_slab.allow_huge_operations=True    
    sc_K_kms = sc_slab#.to(u.K)
    
    
    
    if i ==0:
        vel = np.arange(0,500,2.5)*u.km/u.s
        
        NGC_HCO_Plus = sc_K_kms.unmasked_copy()
    if i==1:
        SiJ2_1 = sc_K_kms.unmasked_copy()

        
del sc_kms #rem from mem
del sc_slab #rem from mem
del sc_K_kms #rem from mem
del subcube #rem from mem
del sc #rem from mem

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
SiJ2_1.allow_huge_operations=True
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem

SiJ2_1_0_reproject = SiJ2_1.reproject(HCOPH,use_memmap=True)  
del SiJ2_1 #rem from mem
del HCOPH
SiJ2_1_reproject_interp = SiJ2_1_0_reproject.spectral_interpolate(spectral_grid=vel) # Match velocities too     
del SiJ2_1_0_reproject
SiJ2_1_reproject_interp.write('SiJ2_1.cube.reproject.Jy-kms.pbcor.fits',overwrite=True)       

del SiJ2_1_reproject_interp        
          
    

    
    
    
print("done")

In [39]:
files=['NGC253.HCO+_4-3_mJy.pbcor.fits','NGC253.HCN_4-3_mJy.pbcor.fits',"NGC253.H13CN_4-3_mJy.pbcor.fits","HCN_J1-0.cube.fits",'HCOp_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits', 'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits',"NGC253.band3.CO_1-0.PB_edge_masked.fits"]     

filen='NGC253.HCO+_4-3_32pc_beam_mJy_Reproject.pbcor.fits'

dub='H13COp_J1-0_SiO_J2-1.cube.fits'    


for i,file in enumerate(files):
    print(file)
    sc = SpectralCube.read(file)    
    if i ==0:                                # Find location of brightest emission
        z,xp,yp = pylab.where(sc.hdu.data==np.nanmax(sc.hdu.data)) 
        yRaxDec = sc.wcs[:][:][0].pixel_to_world(yp,xp)
    y, x = sc.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
    #Need to reduce cube size, but it won't work if theyre all the same size sometimes?
    if i==0 or i ==9:


        cen_p1 = SkyCoord('00h47m33.9s', '-25d17m26.8s', frame='icrs')#SkyCoord('00h47m34.2s', '-25d17m44.8s', frame='icrs')
        cen_p2 = SkyCoord('00h47m32.6s', '-25d17m10.2s', frame='icrs')#SkyCoord('00h47m31.8s', '-25d17m06s', frame='icrs')
        scW = sc.wcs[:][:][0]
        

        raDP1 = [int(scW.world_to_pixel(cen_p1)[0]),int(scW.world_to_pixel(cen_p1)[1])]
        raDP2 = [int(scW.world_to_pixel(cen_p2)[0]),int(scW.world_to_pixel(cen_p2)[1])]
        print(raDP1,raDP2)
        if raDP1[0]<0:
            raDP1[0]=100
        if raDP1[1]<0:
            raDP1[1]=100
        sc = sc#[:,raDP1[1]:raDP2[1],raDP1[0]:raDP2[0]] # Make a subcube



        #Put in the right system
        sc_kms = sc.with_spectral_unit(u.km/u.s,velocity_convention="radio") # Change units from Hz to km/s
        del sc

        sc_slab = sc_kms#.spectral_slab(100. *u.km / u.s, 400. *u.km / u.s)  # Crop out velocities we don't care about    
        del sc_kms
        sc_slab.allow_huge_operations=True    

        sc = sc_slab#.to(u.K)
        del sc_slab
        


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=32*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        print(np.max(sc))
        sc = sc.convolve_to(beam)   
        print(np.max(sc))
        gal="NGC253"
        print(np.shape(sc))
        reheader = sc.header
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader,reheader.cards)
        sc_K_kms = sc.reproject(reheader, order='bilinear', use_memmap=True, filled=True)
        print(np.max(sc_K_kms))
        print("1")
        del sc

        y, x = sc_K_kms.wcs[:][:][0].world_to_pixel(yRaxDec)# Gets the pixel for the same Ra/dec of each cube
        #Need to reduce cube size, but it won't work if theyre all the same size sometimes?

        #print(sc_K_kms.wcs)
        #print(sc_K_kms.world[:,0,0][0])
        #print(header)
    
    
    if i ==0:
            vel = np.arange(0,501,2.5)*u.km/u.s


            fwhm_factor = np.sqrt(8*np.log(2))

            NGC_HCO_reprojectGauss = sc_K_kms.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
            del sc_K_kms
            
            NGC_HCO_Plus_Reproject_Interp = NGC_HCO_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
            del NGC_HCO_reprojectGauss
            
            NGC_HCO_Plus_Reproject_Interp = NGC_HCO_Plus_Reproject_Interp.hdu
            
            NGC_HCO_Plus_Reproject_Interp = SpectralCube.read(NGC_HCO_Plus_Reproject_Interp)


            NGC_HCO_Plus_Reproject_Interp.write(filen,overwrite=True)
            del NGC_HCO_Plus_Reproject_Interp
        
        
    if i==9:
        NGC_CO_J1_0 = sc_K_kms

        

del cen_p1
del cen_p2
del scW
del raDP1
del raDP2
NGC_CO_J1_0.allow_huge_operations=True
NGC_HCO_Plus = sc = SpectralCube.read(filen)
HCOPH = NGC_HCO_Plus.header
HCOPH['CTYPE3'] = 'VRAD'
del NGC_HCO_Plus #rem from mem



fwhm_factor = np.sqrt(8*np.log(2))

CO10_reprojectGauss = NGC_CO_J1_0.spectral_smooth(Gaussian1DKernel(4/fwhm_factor))#I don't know what this is. "This can be useful if you want to interpolate onto a coarser grid but maintain Nyquist sampling. You can then use the spectral_interpolate method to regrid your smoothed spectrum onto a new grid.""
del NGC_CO_J1_0

CO10_reproject_interp = CO10_reprojectGauss.spectral_interpolate(spectral_grid=vel) # Match velocities to -250 250 range 
print(CO10_reproject_interp.world[:][0][0],CO10_reprojectGauss.world[:][0][0])
del CO10_reprojectGauss
CO10_reproject_interp = CO10_reproject_interp.hdu

NGC_CO_J1_0_reproject_interp = SpectralCube.read(CO10_reproject_interp)

NGC_CO_J1_0_reproject_interp.allow_huge_operations=True


NGC_CO_J1_0_reproject = NGC_CO_J1_0_reproject_interp.reproject(HCOPH,use_memmap=True)  
del NGC_CO_J1_0_reproject_interp #rem from mem
del HCOPH


NGC_CO_J1_0_reproject.write('NGC_CO_J1_0.cube.reproject.K-kms.pbcor.fits',overwrite=True)      


del NGC_CO_J1_0_reproject  

print("done")         

[288, 330] [641, 662]
0.0005238471269767526 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=1.8858496571163093 arcsec BMIN=1.8858496571163093 arcsec BPA=0.0 deg
65730.9765625 mJy / beam
0.0 -1.2796754075622423
2.7303551859974873e-07 2.7226215566656354e-07
1082416.9555349313 mJy / beam
(252, 938, 1073)
-0.00010476942539535052 11.895693160611499 B
0.00010476942539535052 -25.29536924799503 c
158 125
1077016.7165507432 mJy / beam
1
[279, 282] [349, 349]
0.0005238471269767526 deg 4.722222222222e-05 3.611111111111e-05
Beam: BMAJ=1.8858496571163093 arcsec BMIN=1.8858496571163093 arcsec BPA=0.0 deg
56.21185302734375 K
0.0 1.3350105633655784
1.3317684107823687e-07 1.7317728615641416e-08
50.97364140160315 K
(141, 640, 640)
-0.00010476942539535052 11.912689225821474 B
0.00010476942539535052 -25.31041396800948 c
469 424
50.68493471298606 K
1
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.

In [40]:
files = pylab.array(['NGC_CO_J1_0.cube.reproject.K-kms.pbcor.fits','ngc253_850_micron_dust.fits'])



for i,file in enumerate(files):
    
    if i ==0:
        subcube = SpectralCube.read(file) 
    
    if i==1:
        sc = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    
    if i ==1:
        sc.allow_huge_operations=True
        scW = sc.wcs



        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=32*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        sc = sc.convolve_to(beam)   
        gal="NGC253"
        print(np.shape(sc))
        reheader = copy.deepcopy(sc.header)
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader)
        scN = sc.reproject(reheader, order='bilinear')
        print("1")
        del sc



        #Put in the right system
        subcube = scN
        del scN
    
    
    if i ==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
        print(np.shape(NGC_HCO_Plus))
    if i==1:
        Cont = subcube
        print(np.shape(Cont))
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header

HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject_32pc_beam.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

(125, 158)
Beam: BMAJ=1.8858496571163093 arcsec BMIN=1.8858496571163093 arcsec BPA=0.0 deg
0.0 -1.2433084728122887
2.732244848151243e-07 2.726697699483849e-07
(1500, 2500)
-0.00010476942539535052 11.895597397647581 B
0.00010476942539535052 -25.29249980148372 c
147 80
1
(80, 147)
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  158                                                  NAXIS2  =                  125                                                  BMAJ    = 0.000523847126976752                                                  BMIN    = 0.000523847126976752                                                  BPA     =                  0.0                                                  BTYPE   = 'Intensity'                                                           

In [ ]:
files = pylab.array(['NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits','ngc253_850_micron_dust.fits'])



for i,file in enumerate(files):
    
    if i ==0:
        subcube = SpectralCube.read(file) 
    
    if i==1:
        sc = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    
    if i ==1:
        sc.allow_huge_operations=True
        scW = sc.wcs


        pathInf = 'NGC253.HCO+_4-3_mJy.pbcor.fits'
        HI = SpectralCube.read(pathInf).header
        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=3*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        print(res,HI["BMAJ"],HI["BMIN"])
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        sc = sc.convolve_to(beam)   
        gal="NGC253"
        print(np.shape(sc))
        reheader = copy.deepcopy(sc.header)
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader)
        scN = sc.reproject(reheader, order='bilinear')
        print("1")
        del sc



        #Put in the right system
        subcube = scN
        del scN
    
    
    if i ==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
        print(np.shape(NGC_HCO_Plus))
    if i==1:
        Cont = subcube
        print(np.shape(Cont))
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header

HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

In [59]:
files = pylab.array(['NGC253_HCO+.cube_5pc_beam.reproject.mJy-kms.pbcor.fits','ngc253_850_micron_dust.fits'])



for i,file in enumerate(files):
    
    if i ==0:
        subcube = SpectralCube.read(file) 
    
    if i==1:
        sc = spectral_cube.Projection.from_hdu(fits.open(file)[0])
    #print(subcube.hdu.header)
    
    
    #Put in the right system
    
    
    if i ==1:
        sc.allow_huge_operations=True
        scW = sc.wcs



        #Nres = (HI["BMAJ"]+HI["BMIN"])/2*3.5*10**6/180*np.pi*u.pc#pc beam res, project everything to same pc res
        Nres=5*u.pc
        dist=3.5*10**6
        res=  Nres/(dist)*180/np.pi*u.deg/u.pc#deg corresponding to ~3 pc
        beam = radio_beam.Beam(major=res, minor=res, pa=0*u.deg)
        print(beam)
        sc.allow_huge_operations=True
        sc = sc.convolve_to(beam)   
        gal="NGC253"
        print(np.shape(sc))
        reheader = copy.deepcopy(sc.header)
        if sc.header['cdelt1']>0:
            pix_x    = (res/5.).to(u.degree).value
            origin_x = sc.longitude_extrema[1].to(u.degree).value if gal=='GC' else sc.longitude_extrema[0].to(u.degree).value
            print(pix_x,origin_x,"A")
        else:
            pix_x    = -1.*(res/5.).to(u.degree).value
            origin_x = (sc.longitude_extrema[0]).to(u.degree).value if gal=='GC' else (sc.longitude_extrema[1]).to(u.degree).value
            print(pix_x,origin_x,"B")
        if sc.header['cdelt2']>0:
            pix_y    = (res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[0].to(u.degree).value
            print(pix_y,origin_y,"c")
        else:
            pix_y    = -1.*(res/5.).to(u.degree).value
            origin_y = sc.latitude_extrema[1].to(u.degree).value
            print(pix_y,origin_y,"d")
        if gal=='NGC253':
            npix_x   = int(np.ceil(np.diff(sc.longitude_extrema, n=1)[0]/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)
        elif gal=='GC':
            npix_x   = int(np.ceil((sc.longitude_extrema[0]-(sc.longitude_extrema[1]-360*u.degree))/np.abs(pix_x)).value)
            npix_y   = int(np.ceil(np.diff(sc.latitude_extrema, n=1)[0]/np.abs(pix_y)).value)
            print(npix_x,npix_y)

        reheader['cdelt1'] = pix_x
        reheader['cdelt2'] = pix_y
        reheader['naxis1'] = npix_x
        reheader['naxis2'] = npix_y
        reheader['crval1'] = origin_x
        reheader['crval2'] = origin_y
        reheader['crpix1'] = 0
        reheader['crpix2'] = 0
        try:
            del reheader['lonpole']
            del reheader['latpole']
            del reheader['wcsaxes']
        except:
            pass

        # regrid cube to target pixel size
        #print(reheader)
        scN = sc.reproject(reheader, order='bilinear')
        print("1")
        del sc



        #Put in the right system
        subcube = scN
        del scN
    
    
    if i ==0:
        vel, dec, ra = subcube.world[:,0,0]            # Grab the velocity axis to use for rebinning
        
        NGC_HCO_Plus = subcube.moment0()
        print(np.shape(NGC_HCO_Plus))
    if i==1:
        Cont = subcube
        print(np.shape(Cont))
        

        
del dec
del ra
Cont.allow_huge_operations=True
#HCOPH = NGC_HCO_Plus.header

HCOPH = NGC_HCO_Plus.hdu.header
#HCOPH['NAXIS'] = 2
print(HCOPH)
#print([HCOPH['NAXIS{0}'.format(i + 1)] for i in range(HCOPH['NAXIS'])][::-1])
print([Cont.hdu.header['NAXIS{0}'.format(i + 1)] for i in range(Cont.hdu.header['NAXIS'])][::-1])
print(np.shape(NGC_HCO_Plus))
del NGC_HCO_Plus #rem from mem

Cont_reproject =Cont.reproject(HCOPH)  
del HCOPH

Cont_reproject.write('Continuum_Reproject_5pc_beam.fits',overwrite=True) 
print(np.shape(Cont_reproject))
del Cont_reproject          
print("done")        

(281, 331)
Beam: BMAJ=0.29466400892442335 arcsec BMIN=0.29466400892442335 arcsec BPA=0.0 deg
0.0 -1.2433084728122887
5.5082771692686565e-09 4.953562302529285e-09
(1500, 2500)
-1.6370222718023522e-05 11.895597397647581 B
1.6370222718023522e-05 -25.29249980148372 c
939 509
1
(509, 939)
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  331                                                  NAXIS2  =                  281                                                  BMAJ    = 8.18511135901176E-05                                                  BMIN    = 8.18511135901176E-05                                                  BPA     =                  0.0                                                  BTYPE   = 'Intensity'                                                      

In [3]:
#Fix frequency info

files = pylab.array(['HCOp_J1-0.cube.fits','HCN_J1-0.cube.fits','H13CN_J1-0.cube.fits','ngc253_co32_hires_common.fits',"SiO_2_1.fits","H13CO_Plus_1_0.fits",'CCH_N1-0.cube.fits','HC3N_J11-10_SO_J45-44.cube.fits','NGC253.H13CN_4-3.pbcor.fits','NGC253.HCO+_4-3.pbcor.fits','NGC253.HCN_4-3.pbcor.fits',"NGC253.band3.CO_1-0.PB_edge_masked.fits"])

filesR = pylab.array(['NGC_HCOP_J1_0.cube.reproject.Jy-kms.pbcor.fits','NGC_HCN_J1.cube.reproject.Jy-kms.pbcor.fits','NGC_H13CN_J1_0.cube.reproject.Jy-kms.pbcor.fits','NGC_CO_J3_2.cube.reproject.Jy-kms.pbcor.fits','SiJ2_1.cube.reproject.Jy-kms.pbcor.fits','H13CoPJ1_0.cube.reproject.Jy-kms.pbcor.fits','CCH_N1.cube.reproject.Jy-kms.pbcor.fits','HC3N_J11.cube.reproject.Jy-kms.pbcor.fits','NGC_H13CN.cube.reproject.mJy-kms.pbcor.fits','NGC253.HCO+_4-3_mJy_Reproject.pbcor.fits','NGC253_HCN.cube.reproject.mJy-kms.pbcor.fits','NGC_CO_J1_0.cube.reproject.K-kms.pbcor.fits'])
for i in range(len(filesR)):
    try:
        freq = fits.getheader(files[i])["RESTFREQ"]
    except:
        freq = fits.getheader(files[i])["RESTFRQ"]
    c=SpectralCube.read(filesR[i])
    ch=c.hdu.header
    try:
        
        ch["RESTFREQ"] =freq
        ch["RESTFRQ"] =freq
        print(ch["RESTFREQ"],"L",ch["RESTFRQ"])
        #print(ch,"DONNNNN1")
    except:
        ch["RESTFRQ"] =freq
        print(ch["RESTFRQ"],"P",ch["RESTFREQ"])
        #print(ch,"DONNNNN")
    
    
    #c.write(filesR[i],overwrite=True)
    
    #CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=ch)*u.K
    #print(ch.cards)
    chp = fits.Header.fromstring(ch.tostring())
    c.allow_huge_operations = True
    CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=chp)
    CN.allow_huge_operations=True
    CN=CN*c[0][0][0].unit
    FN = "Fixed_"+filesR[i]
    print(CN.header["RESTFRQ"],CN.header.cards["RESTFREQ"])
    #CN.header.cards["RESTFREQ"] = CN.header["RESTFRQ"]
    print(CN.header["RESTFRQ"],CN.header["RESTFREQ"])
    #print(CN.wcs)
    CN.write(FN,overwrite=True)
    
CMZpathsR = pylab.array(['cmz_HCOp.cube.mJy-kms.pbcor.fits','cmz_HCN.cube.reproject.mJy-kms.pbcor.fits','cmz_H13CN.cube.reproject.mJy-kms.pbcor.fits',"Spliced_CO_V3.fits",'cmz_H13COp.cube.reproject.mJy-kms.pbcor.fits'])
#CMZpathsR = pylab.array(['cmz_HCOp.cube.mJy-kms.pbcor.fits','cmz_HCN.cube.reproject.mJy-kms.pbcor.fits','cmz_H13CN.cube.reproject.mJy-kms.pbcor.fits',"Spliced_CO_V2.fits",'cmz_H13COp.cube.reproject.mJy-kms.pbcor.fits'])


for i in range(len(CMZpathsR)):
    try:
        freq = fits.getheader(files[i])["RESTFREQ"]
    except:
        freq = fits.getheader(files[i])["RESTFRQ"]
    c=SpectralCube.read(CMZpathsR[i])
    ch=c.hdu.header
    try:
        
        ch["RESTFREQ"] =freq
        ch["RESTFRQ"] =freq
        print(ch["RESTFREQ"],"L",ch["RESTFRQ"])
    except:
        ch["RESTFRQ"] =freq
        print(ch["RESTFRQ"],"P",ch["RESTFREQ"])
        
    
    
    #c.write(filesR[i],overwrite=True)
    
    CN =  SpectralCube(data=c.hdu.data, wcs=c.wcs,header=ch)*c[0][0][0].unit
    FN = "Fixed_"+CMZpathsR[i]
    print("C",CN.header["RESTFRQ"],CN.header["RESTFREQ"])
    CN.write(FN,overwrite=True)
    
    




89188523000.0 L 89188523000.0
356734220000.0 RESTFREQ=        89188523000.0                                                  
356734220000.0 89188523000.0
88631847000.0 L 88631847000.0
356734220000.0 RESTFREQ=        88631847000.0                                                  
356734220000.0 88631847000.0
86338767000.0 L 86338767000.0
356734220000.0 RESTFREQ=        86338767000.0                                                  
356734220000.0 86338767000.0
345795990000.0 L 345795990000.0
356734220000.0 RESTFREQ=       345795990000.0                                                  
356734220000.0 345795990000.0
86754288000.0 L 86754288000.0
356734220000.0 RESTFREQ=        86754288000.0                                                  
356734220000.0 86754288000.0
86754288000.0 L 86754288000.0
356734220000.0 RESTFREQ=        86754288000.0                                                  
356734220000.0 86754288000.0
87316925000.0 L 87316925000.0
356734220000.0 RESTFREQ=        87316